# Assignment 8: Automated Machine Learning (Part 2)
## Objective:

As we learned from the class, the high demand for machine learning has produced a large amount of data scientists who have developed expertise in tools and algorithms. The features in the data will directly influence the results. However, it is tedious and unscalable to manually design and select features without domain knowledge. Thus, using some AutoML techniques will significantly help data scientists save labour and time. 
After completing this assignment, you should be able to answer the following questions:

1. Why do we need AutoML?
2. How does auto feature generation work?
3. How to use featuretools library to automatically generate features?
4. How to get useful features in a large feature space?

Imagine you are a data scientist in an online retailer company, for example, Amazon. Your task is to provide some recommendations to customers based on their historical purchase records.

In this assignment, we predict whether the customer will buy **Banana** in the next 4 weeks. It is a classification problem. To simplify the problem, we have already generated some features and provide the accuracy of the model (Random Forest model). The task for you is to generate **10** useful features and beat our model performance (AUC = 0.61, see below). 

For example, <br>
`MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas` means whether the most frequent purchase of the customer is Bag of Organic Bananas. 

```
1: Feature: MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas
2: Feature: MODE(order_products.aisle_id) is unknown
3: Feature: SUM(orders.NUM_UNIQUE(order_products.product_name))
4: Feature: MODE(orders.MODE(order_products.product_name)) = Boneless Skinless Chicken Breasts
5: Feature: MODE(order_products.product_name) = Boneless Skinless Chicken Breasts
6: Feature: STD(orders.NUM_UNIQUE(order_products.aisle_id))
7: Feature: MODE(order_products.aisle_id) = 83
8: Feature: MEDIAN(orders.MINUTE(order_time))
9: Feature: MODE(orders.DAY(order_time)) = 23
10: Feature: MODE(orders.MODE(order_products.department)) = produce

AUC 0.61
```


## Preliminary
If you never use featuretools before, you need to learn some basic knowledge of this topic. 
I found that these are some good resources: 
* [featuretools documentation](https://docs.featuretools.com/en/stable/)
* [Tutorial: Automated Feature Engineering in Python](https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219)

The data can be downloaded from [A8-2-data.zip](A7-2-data.zip). 

## 0. Preparation
Import relevant libraries and load the dataset: <br>
users: <br>
* user_id: customer identifier
* label:  1 if the customer will buy banana in next 4 weeks, 0 otherwise

orders: <br>
* order_id: order identifier
* user_id: customer identifier
* order_time: date of the order was placed on 

order_products: <br>
* order_id: order identifier
* order_product_id: foreign key
* reordered:  1 if this product has been ordered by this user in the past, 0 otherwise
* product_name: name of the product
* aisle_id: aisle identifier
* department: the name of the department
* order_time: date of the order was placed on

In [21]:
# Import Statements
import pandas as pd
import featuretools as ft
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import os
ft.__version__

'1.6.0'

In [22]:
# Import the data
orders = pd.read_csv("orders.csv")
order_products = pd.read_csv("order_products.csv")
users = pd.read_csv("users.csv")

print(users["label"].value_counts())
orders.shape

False    628
True     139
Name: label, dtype: int64


(5997, 4)

## Task 1. Feature Generation
In this task, you need to use featuretools to generate candidate features by using the above three tables.

### 1.1 Representing Data with EntitySet

Define entities and their relationships (see [https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html](https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html))

In [24]:
# Get the relationship between entities
def load_entityset(orders, order_products, users):
    # --- Write your code below ---
    # Specifying dictionary of dataframes
    dataframes = {
        "users" : (users, "user_id"),
        "orders" : (orders, "order_id", "order_time"),
        "order_products" : (order_products, "order_product_id", "order_time")
    }
    
    # Establishing the relationships among dataframes
    relationships = [("users", "user_id", "orders", "user_id"),
                     ("orders", "order_id", "order_products", "order_id")]
    
    # Return the EntitySet object
    return ft.EntitySet("cust-entity-set", dataframes, relationships)

### 1.2 Deep Feature Synthesis

In [25]:
# Automatically generate features
es = load_entityset(orders, order_products, users)

# use ft.dfs to perform feature engineering
# --- Write your code below ---
feature_matrix, feature_defs = ft.dfs(entityset=es, target_dataframe_name="users")

In [26]:
# output what features you generate
feature_matrix

,Unnamed: 0,label,COUNT(orders),MAX(orders.Unnamed: 0),MEAN(orders.Unnamed: 0),MIN(orders.Unnamed: 0),SKEW(orders.Unnamed: 0),STD(orders.Unnamed: 0),SUM(orders.Unnamed: 0),COUNT(order_products),...,SUM(orders.SKEW(order_products.reordered)),SUM(orders.STD(order_products.Unnamed: 0)),SUM(orders.STD(order_products.aisle_id)),SUM(orders.STD(order_products.reordered)),MAX(order_products.orders.Unnamed: 0),MEAN(order_products.orders.Unnamed: 0),MIN(order_products.orders.Unnamed: 0),SKEW(order_products.orders.Unnamed: 0),STD(order_products.orders.Unnamed: 0),SUM(order_products.orders.Unnamed: 0)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,False,4,3.0,1.5,0.0,0.0,1.290994,6.0,21,...,-0.608581,6.614245,160.426417,1.095445,3.0,1.476190,0.0,0.065417,1.123345,31.0
2,1,True,7,17.0,14.0,11.0,0.0,2.160247,98.0,85,...,7.175015,25.523426,261.248415,2.443579,17.0,14.482353,11.0,-0.517595,2.038976,1231.0
3,2,False,5,30.0,28.0,26.0,0.0,1.581139,140.0,41,...,2.432523,12.534855,208.749499,1.999461,30.0,27.951220,26.0,0.125825,1.564468,1146.0
7,3,False,4,57.0,55.5,54.0,0.0,1.290994,222.0,73,...,-1.487708,21.642408,142.991508,1.230281,57.0,55.602740,54.0,-0.122608,1.010222,4059.0
10,4,False,4,86.0,84.5,83.0,0.0,1.290994,338.0,114,...,6.950682,33.476705,146.689346,1.074301,86.0,84.771930,83.0,0.088343,0.892794,9664.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,762,False,4,16051.0,16049.5,16048.0,0.0,1.290994,64198.0,45,...,-0.967009,13.554881,180.136629,1.489449,16051.0,16049.444444,16048.0,0.095587,1.216469,722225.0
997,763,False,4,16056.0,16054.5,16053.0,0.0,1.290994,64218.0,33,...,1.981310,10.084368,138.419278,0.963430,16056.0,16054.939394,16053.0,-0.648491,1.087985,529813.0
998,764,False,7,16077.0,16074.0,16071.0,0.0,2.160247,112518.0,60,...,-3.921700,18.292581,239.109143,1.308153,16077.0,16074.200000,16071.0,-0.277419,1.754413,964452.0


## Task 2. Feature Selection
In this task, you are going to select 10 features that are useful and train the *Random Forest* model. The goal is to beat the accuracy performance as we have shown before. Note that you have to use the Random Forest and the hyperparameters we provide in Section 2.2. In other words, your job is to achieve a higher AUC than 0.61 through feature generation/selection rather than through hyperparameter tuning or model selectoin. 

### 2.1 Select top features

In [27]:
# --- Write your code below ---
# Select top-10 features and return X, y (X.shape = (767, 10))
from sklearn.feature_selection import SelectKBest, f_classif

# Encoding the categorical columns
fm_encoded, f_encoded = ft.encode_features(feature_matrix, feature_defs)

# Input data
X = fm_encoded.drop(['label'], axis=1)
X = X.fillna(0)

# Input label
y = fm_encoded['label']

# Selecting the best 10 features 
kbest = SelectKBest(f_classif, k=10).fit(X, y)
top_features = X.columns[kbest.get_support()]
X = X[top_features]

C:\Users\Elavarasan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:115: UserWarning: Features [  5  17  38  59  71  74  77  85  91 122 139 209 217 218 219] are constant.
  UserWarning)
C:\Users\Elavarasan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


### 2.2 Get accuracy and list features

In [31]:
clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)
scores = cross_val_score(estimator=clf,X=X, y=y, cv=3,
                             scoring="roc_auc", verbose=True)

print("AUC %.2f" % (scores.mean()))

# Print top-10 features
print("\nTop 10 features are : ", top_features.values)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


AUC 0.76

Top 10 features are :  ['NUM_UNIQUE(order_products.department)'
 'NUM_UNIQUE(order_products.product_name)'
 'MEAN(orders.COUNT(order_products))'
 'MEAN(orders.NUM_UNIQUE(order_products.department))'
 'MEAN(orders.NUM_UNIQUE(order_products.product_name))'
 'MEAN(orders.STD(order_products.Unnamed: 0))'
 'MEAN(orders.SUM(order_products.aisle_id))'
 'MODE(order_products.product_name) = Banana'
 'MODE(order_products.product_name) is unknown'
 'MODE(orders.MODE(order_products.product_name)) = Banana']


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.7s finished


## Task3. Writing Questions

1. Please list three advantages and disadvantages of featuretools. 
2. For those disadvantages you listed above, do you have any idea to improve it? 

# Answers

1. Advantages:
    - Allows to setup relationships between entities if exists.
    - Automatically generates complex features and improves the predictive performance of the model.
    - Makes feature engineering faster and easier which reduces the development time.
    
   Disadvantages:
    - It does not work on simple data set or where there is no parent/child relationship.
    - Feature tools can run only data sets that can fit in memory on one machine.
    
    
2. If there is no relationships between datasets(in some cases) then it is better to use other feature engineering techniques. Increase memory to fit the data such that feature tool can be applied.

## Submission
Complete the code in this notebook, and submit it to the CourSys activity Assignment 8.